# Missing Data

Manejar los datos faltantes en una base de datos es crucial por varias razones, ya que los datos faltantes pueden afectar significativamente la calidad de los análisis y los resultados de los modelos de machine learning.

## 1. Eliminacion de valores faltantes

Se usa de manera comoda cuando la cantidad de datos faltantes es pequeña y la eliminación no afectará significativamente el análisis.

In [21]:
import pandas as pd

# Datos de ejemplo
data = pd.DataFrame({
    'A': [1, 2, None, 4],
    'B': [None, 2, 3, 4],
    'C': [1, None, None, 4]
})

# Eliminar filas con valores faltantes
data_drop_rows = data.dropna()

# Eliminar columnas con valores faltantes
data_drop_cols = data.dropna(axis=1)

print("Filas sin valores faltantes:\n", data_drop_rows)
print("Columnas sin valores faltantes:\n", data_drop_cols)


Filas sin valores faltantes:
      A    B    C
3  4.0  4.0  4.0
Columnas sin valores faltantes:
 Empty DataFrame
Columns: []
Index: [0, 1, 2, 3]


## 2. Imputacion simple

Se llenan los datos faltantes con alguna estadistica simple como la media, moda o mediana. Se puede hacer estratificacion o filtrado para dar mayor precisión


In [23]:
from sklearn.impute import SimpleImputer

# Imputación con la media a toda la base
imputer_mean = SimpleImputer(strategy='mean') # Reemplaza mean con median o most_frequent
data_mean_imputed = imputer_mean.fit_transform(data)

# Imputación con la media solo a ciertas columnas
imputer_mean_1 = SimpleImputer(strategy='mean') # Reemplaza mean con median o most_frequent
data_mean_imputed_1 = imputer_mean_1.fit_transform(data.iloc[:, 0:1])

print("Datos imputados con la media:\n", data_mean_imputed)
print("Datos imputados con la media solo a una columna:\n", data_mean_imputed_1)


Datos imputados con la media:
 [[1.         3.         1.        ]
 [2.         2.         2.5       ]
 [2.33333333 3.         2.5       ]
 [4.         4.         4.        ]]
Datos imputados con la mediana:
 [[1.        ]
 [2.        ]
 [2.33333333]
 [4.        ]]


## 3. Imputación con Valores Constantes

Se puede usar cuando existe un valor específico que tiene sentido para imputar los valores faltantes, como 0 para características numéricas o 'missing' para categóricas.

In [ ]:
# Imputación con un valor constante
imputer_constant = SimpleImputer(strategy='constant', fill_value=0)
data_constant_imputed = imputer_constant.fit_transform(data)

print("Datos imputados con un valor constante:\n", data_constant_imputed)


## 4. Imputación Basada en Vecinos (KNN Imputation)

Se puede usar cuando la relación entre características es compleja y puede ser mejor capturada por los vecinos más cercanos.

In [ ]:
from sklearn.impute import KNNImputer

# Imputación basada en KNN
imputer_knn = KNNImputer(n_neighbors=2)
data_knn_imputed = imputer_knn.fit_transform(data)

print("Datos imputados con KNN:\n", data_knn_imputed)


## 5. Imputación con Modelos Predictivos

Se puede usar cuando se desea utilizar la información de otras características para predecir los valores faltantes. Aqui se pueden usar diferentes modelos de aprendizaje de máquina.

In [ ]:
from sklearn.linear_model import LinearRegression

# Supongamos que queremos imputar la columna 'A' usando las columnas 'B' y 'C'
train = data.dropna(subset=['A'])
predict = data[data['A'].isna()]

X_train = train[['B', 'C']]
y_train = train['A']
X_predict = predict[['B', 'C']]

# Entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir los valores faltantes
data.loc[data['A'].isna(), 'A'] = model.predict(X_predict)

print("Datos imputados con un modelo predictivo:\n", data)


## 6. Interpolación

Se puede usar cuando los valores faltantes siguen un patrón temporal o secuencial. Existen diferentes metodos, cada uno se adapta mejor a cierto patron.

- Interpolación Lineal: Adecuada para datos que se espera que cambien de manera constante y gradual. Es útil para datos temporales o espaciales con intervalos regulares y cambios suaves.

- Interpolación Polinómica: Útil cuando los datos tienen una relación no lineal, pero puede ser sensible a los outliers y puede producir oscilaciones no deseadas para polinomios de orden alto.

- Interpolación Spline: Adecuada para datos que requieren una interpolación suave y continua. Es ideal para datos temporales o espaciales con cambios suaves pero no necesariamente lineales.

In [ ]:
# Interpolación lineal
data_interpolated = data.interpolate()

print("Datos interpolados:\n", data_interpolated)


## 7. Imputación por Forward Fill y Backward Fill

Se puede usar cuando los datos tienen un orden natural y se puede asumir que los valores faltantes son similares a los valores anteriores o posteriores.

In [ ]:
# Forward fill
data_ffill = data.ffill()

# Backward fill
data_bfill = data.bfill()

print("Datos con forward fill:\n", data_ffill)
print("Datos con backward fill:\n", data_bfill)


## 8. Uso de Variables Indicadoras

Se puede usar para conservar información sobre la falta de datos, creando una nueva columna que indique si el valor era originalmente faltante.

In [ ]:
# Crear una variable indicadora para la columna 'A'
data['A_missing'] = data['A'].isna().astype(int)

# Imputar la columna 'A' con la media
imputer = SimpleImputer(strategy='mean')
data['A'] = imputer.fit_transform(data[['A']])

print("Datos con variable indicadora y imputación:\n", data)


## 9. Multiple Imputation by Chained Equations (MICE) 

La **Imputación Múltiple mediante Ecuaciones Encadenadas**, es una técnica para manejar valores faltantes en conjuntos de datos. A diferencia de los métodos simples de imputación (como reemplazar los valores faltantes por la media o la mediana), MICE imputa los valores faltantes múltiples veces, lo que genera varias versiones del conjunto de datos con diferentes valores imputados. Luego, los resultados se combinan para dar una estimación más robusta y precisa.

### Metodología

El proceso de MICE consiste en realizar una imputación iterativa para cada valor faltante, utilizando un modelo adecuado para cada variable que tenga datos faltantes. El algoritmo sigue estos pasos:

1. **Imputación Inicial**: Se realiza una imputación inicial para los valores faltantes (puede ser con la media, mediana u otra técnica).
2. **Modelos por Variable**: Para cada variable con valores faltantes, se ajusta un modelo predictivo en función de las demás variables. Este modelo se usa para imputar los valores faltantes de esa variable.
3. **Imputación Iterativa**: El proceso se repite iterativamente para cada variable hasta que se estabilizan los valores imputados.
4. **Imputaciones Múltiples**: Se repite el proceso varias veces para obtener diferentes conjuntos de datos completos, con el fin de reflejar la incertidumbre asociada a los valores imputados.
5. **Combinación de Resultados**: Los análisis se realizan en cada conjunto de datos imputado, y los resultados se combinan para obtener inferencias finales.

### Ventajas de MICE
- **Múltiples Imputaciones**: Al hacer varias imputaciones, MICE captura la variabilidad en los datos imputados, lo que da lugar a estimaciones más robustas.
- **Modelos Flexibles**: Cada variable puede tener su propio modelo de imputación, lo que permite ajustar modelos complejos para diferentes tipos de variables (continuas, categóricas, etc.).

### Ejemplo en Python con `fancyimpute`

A continuación, un ejemplo de cómo usar MICE en Python con la biblioteca `fancyimpute` o `IterativeImputer` de `scikit-learn`.

In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Necesario para usar IterativeImputer
from sklearn.impute import IterativeImputer

# Crear un conjunto de datos con valores faltantes
np.random.seed(42)
data = pd.DataFrame({
    'A': [1, 2, np.nan, 4, 5],
    'B': [5, np.nan, np.nan, 8, 10],
    'C': [10, 20, 30, np.nan, 50]
})

print("Datos originales con valores faltantes:")
print(data)

# Inicializar el imputador MICE
mice_imputer = IterativeImputer(max_iter=10, random_state=0)

# Ajustar el imputador y transformar los datos
imputed_data = mice_imputer.fit_transform(data)

# Convertir el resultado a DataFrame para visualizarlo mejor
imputed_data_df = pd.DataFrame(imputed_data, columns=data.columns)

print("\nDatos imputados:")
print(imputed_data_df)